In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns


from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

In [ ]:
train = pd.read_csv("../input/tabhack10/Week5_train.csv", index_col = 'id')
test = pd.read_csv("../input/tabhack10/Week5_test.csv", index_col = 'id')

In [ ]:
# print(test_df.shape)
train.shape, test.shape

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.columns

In [ ]:
train.describe()

In [ ]:
test.isna().sum()

In [ ]:
train.dtypes

In [ ]:
corr_matrix = train.corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
fig=plt.figure(figsize=(16, 8))
sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
plt.figure(figsize=(15,12))
sns.heatmap(train.corr())
plt.show()

In [ ]:
target = train.pop('earning')

In [ ]:
fig=plt.figure(figsize=(6, 6))
sns.countplot(x=target)
plt.show()

In [ ]:
#drop_col = ['id','Working_class', 'education', 'marital_status', 'Occupation', 'relationship', 'race', 'gender', 'native_country']

In [ ]:
test.nunique()
#train[cat_col].nunique()

In [ ]:
train.head(50)
#df['transformed'] = encoder.fit_transform(df)
#oe.fit_transform(train)

In [ ]:
train.shape, test.shape

In [ ]:
train.Working_class.unique()

In [ ]:
train.Working_class.value_counts()

In [ ]:
train.education.unique()

In [ ]:
train.marital_status.unique()

In [ ]:
train.marital_status.value_counts()

In [ ]:
train.Occupation.unique()

In [ ]:
train.Occupation.value_counts()

In [ ]:
train.relationship.unique()

In [ ]:
train.relationship.value_counts()

In [ ]:
train.race.unique()

In [ ]:
train.native_country.unique()

In [ ]:
train.native_country.value_counts()

In [ ]:
train['native_country'] = train['native_country'].str.strip()
test['native_country'] = test['native_country'].str.strip()

In [ ]:
asia = ['Philippines','India', 'China', 'Vietnam', 'Japan','Taiwan','Thailand', 'Cambodia','Hong', 'Laos']
N_america = ['United-States', '?', 'Mexico', 'Canada']
central_am = ['Puerto-Rico','El-Salvador','Dominican-Republic','Columbia','Guatemala','Haiti','Ecuador','Peru','Nicaragua','Honduras','Outlying-US(Guam-USVI-etc)','Trinadad&Tobago','Jamaica','South']
europe = ['Germany','England','Cuba','Italy','Poland','Iran','Ireland','France','Greece','Portugal','Hungary','Scotland','Yugoslavia']

train['native_country'] = train['native_country'].replace(dict.fromkeys(asia, 'asia'))
train['native_country'] = train['native_country'].replace(dict.fromkeys(N_america, 'north_america'))
train['native_country'] = train['native_country'].replace(dict.fromkeys(central_am, 'central_america'))
train['native_country'] = train['native_country'].replace(dict.fromkeys(europe, 'europe'))

test['native_country'] = test['native_country'].replace(dict.fromkeys(asia, 'asia'))
test['native_country'] = test['native_country'].replace(dict.fromkeys(N_america, 'north_america'))
test['native_country'] = test['native_country'].replace(dict.fromkeys(central_am, 'central_america'))
test['native_country'] = test['native_country'].replace(dict.fromkeys(europe, 'europe'))

In [ ]:
train.native_country.unique()

In [ ]:
train.native_country.value_counts()

In [ ]:
#train.drop(columns = ['native_country','Occupation'], axis = 1, inplace = True)

In [ ]:
train.head(50)

In [ ]:
school = [' 7th-8th', ' 12th', ' 5th-6th',
       ' 11th', ' 1st-4th', ' 9th', ' 10th']
#train['education'] = train['education'].str.replace(school, "school")#.astype('str')
train['education'] = train.education.replace(dict.fromkeys(school, 'school'))
test['education'] = test.education.replace(dict.fromkeys(school, 'school'))

In [ ]:
train['Working_class'] = train['Working_class'].replace(" ?", "unempl")
test['Working_class'] = test['Working_class'].replace(" ?", "unempl")
#train['Occupation'] = train['Occupation'].str.replace("?", "unocc")

In [ ]:
work_class_gov = [' Local-gov',' State-gov', ' Federal-gov']
self = [' Self-emp-not-inc', ' Self-emp-inc']
non =       [' Without-pay', 'unempl']

train['Working_class'] = train['Working_class'].replace(dict.fromkeys(work_class_gov, 'gov'))
train['Working_class'] = train['Working_class'].replace(dict.fromkeys(self, 'self_emp'))
train['Working_class'] = train['Working_class'].replace(dict.fromkeys(non, 'no_pay'))

test['Working_class'] = test['Working_class'].replace(dict.fromkeys(work_class_gov, 'gov'))
test['Working_class'] = test['Working_class'].replace(dict.fromkeys(self, 'self_emp'))
test['Working_class'] = test['Working_class'].replace(dict.fromkeys(non, 'no_pay'))

In [ ]:
cat_col = [c for c in train.columns if train.dtypes[c] == 'object']
num_col = [c for c in train.columns if train.dtypes[c] == 'int64']
all_col = cat_col + num_col

In [ ]:
mapp_wc = {'no_pay':0,
           'self_emp':1,
           ' Private':2,
           'gov':3}

mapp_ed = {' Bachelors':4, ' Assoc-voc':6, ' HS-grad':3, ' Masters':5,
       ' Some-college':4, 'school':1, ' Assoc-acdm':6, ' Prof-school':5,
       ' Doctorate':5, ' Preschool':0}

mapp_marital = {' Married-civ-spouse':4, ' Never-married':3, ' Divorced':1, ' Widowed':1,
       ' Separated':1, ' Married-spouse-absent':2, ' Married-AF-spouse':5}

mapp_occup = {' Handlers-cleaners':1, ' Exec-managerial':5, ' Prof-specialty':5,
       ' Tech-support':2, ' Sales':3, ' Machine-op-inspct':2, ' ?':0,
       ' Other-service':4, ' Adm-clerical':3, ' Transport-moving':1,
       ' Farming-fishing':1, ' Craft-repair':1, ' Protective-serv':6,
       ' Priv-house-serv':6, ' Armed-Forces':7}

train['map_workclass'] = train['Working_class'].replace(mapp_wc)
train['map_education'] = train['education'].replace(mapp_ed)
train['map_marital'] = train['marital_status'].replace(mapp_marital)
train['map_occupation'] = train['Occupation'].replace(mapp_occup)


test['map_workclass'] = test['Working_class'].replace(mapp_wc)
test['map_education'] = test['education'].replace(mapp_ed)
test['map_marital'] = test['marital_status'].replace(mapp_marital)
test['map_occupation'] = test['Occupation'].replace(mapp_occup)

In [ ]:
train

In [ ]:
train.drop(columns = ['Working_class','education','marital_status','Occupation'], axis = 1, inplace = True)
test.drop(columns = ['Working_class','education','Occupation','marital_status'], axis = 1, inplace = True)

In [ ]:
ohe_cols = ['relationship','race','gender','native_country']

In [ ]:
#ohe = OneHotEncoder()
#ohe.fit_transform(ohe_cols)
train = pd.get_dummies(train, columns=ohe_cols)#['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex'])
test = pd.get_dummies(test, columns=ohe_cols)

In [ ]:
train.head()

In [ ]:
#train1 = train.copy()
#scaler = StandardScaler()
#train1 = scaler.fit_transform(train1)
#test1 = scaler.transform(test)

In [ ]:
#train1

### Performing the Train Test Split

In [ ]:
X = train#train_df.drop(['earning'], axis=1)
y = target#train_df['earning']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.1)

In [ ]:
print(y_train.value_counts())


In [ ]:
y_val.value_counts()

### Training

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from xgboost import plot_tree, plot_importance
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve

In [ ]:
#train1 = train.copy()
scaler = StandardScaler()
x = pd.DataFrame(scaler.fit_transform(train), columns = train.columns, index = train.index)
#x = scaler.fit_transform(X_train)
x_val = scaler.transform(X_val)
#test1 = scaler.transform(test)

In [ ]:
lr = LogisticRegression()
lr.fit(x,y)

In [ ]:
y_pred = lr.predict(x_val)
score = f1_score(y_val, y_pred)
print(score)

In [ ]:
!pip install flaml

In [ ]:
#cv = cross_val_score(lgbm,x,y_train,cv =5)
#cv.mean()

In [ ]:
from flaml.default import RandomForestClassifier, LGBMClassifier, XGBClassifier

In [ ]:
rf = RandomForestClassifier()
rf.fit(x,y)

In [ ]:
y_pred = rf.predict(x_val)
score = f1_score(y_val, y_pred)
print(score)

In [ ]:
#xgb = XGBClassifier()
#xgb.fit(x,y_train)

In [ ]:
lgbm = LGBMClassifier()
lgbm.fit(x,y)

In [ ]:
#automl = AutoML(task = 'classification')# estimator_list = ['lgbm'])

In [ ]:
#automl.fit(x,y_train)

In [ ]:
#xgbrf = XGBClassifier (gamma= 0.01, learning_rate= 0.001, max_depth= 3, n_estimators= 100)
#xgbrf.fit(x,y_train)

In [ ]:
y_pred = lgbm.predict(x_val)
score = f1_score(y_val, y_pred)
print(score)

In [ ]:
#from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits=5, shuffle=True)
predicts = []
for train_index, test_index in kf.split(x, y):
    X_train, X_val = x.iloc[train_index], x.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    lgbm.fit(X_train, y_train, eval_set=[(X_val, y_val)], 
            early_stopping_rounds=20)
    predicts.append(lgbm.predict(test_sc))

In [ ]:
#p = pd.DataFrame(np.array(predicts).mean(axis=0).round().astype('int'))

In [ ]:
#output = pd.DataFrame({'id': test.index,'earning': p.loc[:,0]})
#output.to_csv('./sub_lgbm_skf.csv', index=False)

In [ ]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(x,y)

In [ ]:
from sklearn.metrics import accuracy_score
score = accuracy_score
#print(score(p,y))

In [ ]:
pgb=svc.predict(x)
pgb
print(score(pgb,y))

### Making Prediction

In [ ]:
test = test.copy()

In [ ]:
test.shape

In [ ]:
test.head()

In [ ]:
#test_data.drop(drop_col, axis=1, inplace=True)

In [ ]:
#test_scale = scaler.transform(test_data)
test_sc = pd.DataFrame(scaler.transform(test), columns = test.columns, index = test.index)

In [ ]:
preds = lgbm.predict(test_sc)

### Submission File

In [ ]:
output = pd.DataFrame({'id': test.index,'earning': preds})
output.to_csv('./sub_lgbm_1.csv', index=False)

In [ ]:
#result = pd.DataFrame(test_data1['id'], columns = ['id'])
#result['earning']= y_test